In [25]:
import tensorflow as tf

In [26]:
class ModelBuilder(object):

    def __init__(self, model_id):
        self.model_id = model_id

    def get_input(self, mode):
        """Implement in derived class"""
        raise NotImplemented()

    def get_train_inputs(self):
        return self.get_input(tf.estimator.ModeKeys.TRAIN)

    def get_predict_inputs(self):
        return self.get_input(tf.estimator.ModeKeys.PREDICT)

    def base_model(self, features, mode):
        """Implemented in the derived class
        """
        raise NotImplemented()

    def get_train_op(self, loss, step):
        raise NotImplemented()
    
    def get_loss(self, predictions, labels):
        raise NotImplemented()

    def build_estimator(self, features, labels, mode, config=None):
        """This method will build the estimator"""
        predictions = self.base_model(features, mode)
        network_arguments = dict(mode=mode, predictions=predictions)

        if mode == tf.estimator.ModeKeys.PREDICT:
            """Execution pipeline"""
            return tf.estimator.EstimatorSpec(**network_arguments)

        
        loss = self.get_loss(predictions, labels)
        network_arguments['loss'] = loss
        
        tf.summary.scalar('loss', loss, family='sublosses')
        
        if mode == tf.estimator.ModeKeys.EVAL:
            return tf.estimator.EstimatorSpec(**network_arguments)

        """Will provide current step. In this case the epoch"""
        step = tf.train.get_or_create_global_step()
        """Collection of operations. These are the operations
        that are to be executed after training step"""
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

        with tf.control_dependencies(update_ops):
            train_op = self.get_train_op(loss = loss,
                                         step = step)

        network_arguments['train_op'] = train_op

        if mode == tf.estimator.ModeKeys.TRAIN:
            return tf.estimator.EstimatorSpec(**network_arguments)

    def get_estimator(self, config=None):
        """Get the `tf.estimator.Estimator` defined by this builder."""
        return tf.estimator.Estimator(
            self.build_estimator, self.model_dir, config=config)

    def train(self, config=None, **train_kwargs):
        """Wrapper around `tf.estimator.Estimator.train`."""
        estimator = self.get_estimator(config=config)
        estimator.train(self.get_train_inputs, **train_kwargs)

    def evaluate(self, config=None, **train_kwargs):
        """Wrapper around `tf.estimator.Estimator.train`."""
        estimator = self.get_estimator(config=config)
        estimator.evaluate(self.get_predict_inputs, **train_kwargs)

In [27]:
import os

from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

def initialize_datasets():
    data = fetch_olivetti_faces()
    targets = data.target
    features = data.images
    encoder = OneHotEncoder()
    targets = encoder.fit_transform(targets.reshape(-1, 1)).todense()
    xTrain, xTest, yTrain, yTest = train_test_split(features, targets, test_size=0.1)

    return {'train':{'features': xTrain, 'labels' : yTrain}, 'test':{'features': xTest, 'labels' : yTest}}

class Classifier(ModelBuilder):
    
    
    def __init__(self, model_id):
        super(Classifier, self).__init__(model_id)
        self.data_dictionary = initialize_datasets()
    
    def make_dataset(self, features, labels):
        dataset = tf.data.Dataset.from_tensor_slices((features, labels))
        return dataset
    
    def get_input(self, mode):
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            dataset = self.make_dataset(self.data_dictionary['train']['features'].reshape(360, -1),
                                   self.data_dictionary['train']['labels'].reshape(360, -1))
        else:
            dataset = self.make_dataset(self.data_dictionary['test']['features'],
                                   self.data_dictionary['test']['labels'])
        
        dataset = dataset.repeat()
        dataset = dataset.prefetch(2)

        return dataset.make_one_shot_iterator().get_next()
    
    def base_model(self, features, mode):
        features = tf.reshape(features, [1, 64 * 64])
        predictions = tf.nn.softmax(tf.matmul(features, tf.Variable(tf.zeros([64 * 64, 40]))) + tf.Variable(tf.zeros([40])))
        
        return predictions
    
    def get_loss(self, predictions, labels):
        labels = tf.cast(labels, tf.float32)
        loss_function = tf.reduce_mean(-tf.reduce_sum(labels * tf.log(predictions), reduction_indices=1))
        return loss_function
    
    def get_train_op(self, loss, step):
        optimizer = tf.train.AdamOptimizer()
        return optimizer.minimize(loss, step)
    
    @property
    def model_dir(self):
        return os.path.join(os.getcwd(), "model", self.model_id)

In [28]:
classifier = Classifier("12")
run_config = tf.estimator.RunConfig(**{'log_step_count_steps':25, 'save_summary_steps':25,
                                       'save_checkpoints_secs':3})
classifier.train(max_steps=5000, config = run_config)

INFO:tensorflow:Using config: {'_model_dir': '/home/saurabh/Documents/tfMulti/model/12', '_tf_random_seed': None, '_save_summary_steps': 25, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 3, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 25, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcedd94ba20>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.


/home/saurabh/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /home/saurabh/Documents/tfMulti/model/12/model.ckpt.
INFO:tensorflow:loss = 3.6888795, step = 1
INFO:tensorflow:global_step/sec: 230.959
INFO:tensorflow:loss = 13.6486025, step = 26 (0.109 sec)
INFO:tensorflow:global_step/sec: 912.107
INFO:tensorflow:loss = 12.960297, step = 51 (0.028 sec)
INFO:tensorflow:global_step/sec: 1046.65
INFO:tensorflow:loss = 9.356352, step = 76 (0.024 sec)
INFO:tensorflow:global_step/sec: 986.162
INFO:tensorflow:loss = 5.5029964, step = 101 (0.026 sec)
INFO:tensorflow:global_step/sec: 712.6
INFO:tensorflow:loss = 9.0983095, step = 126 (0.036 sec)
INFO:tensorflow:global_step/sec: 991.852
INFO:tensorflow:loss = 4.403443, step = 151 (0.025 sec)
INFO:tensorflow:global_step/sec: 796.748
INFO:tensorflow:loss = 7.674576, step = 176 (0.032 sec)
INFO:tensorflow:global_step/sec: 739.888
INFO:tensorflow:lo

INFO:tensorflow:global_step/sec: 712.362
INFO:tensorflow:loss = 0.004466061, step = 2051 (0.035 sec)
INFO:tensorflow:global_step/sec: 868.386
INFO:tensorflow:loss = 1.2047873, step = 2076 (0.029 sec)
INFO:tensorflow:global_step/sec: 842.737
INFO:tensorflow:loss = 15.355498, step = 2101 (0.029 sec)
INFO:tensorflow:global_step/sec: 892.519
INFO:tensorflow:loss = 0.0025489004, step = 2126 (0.028 sec)
INFO:tensorflow:global_step/sec: 698.827
INFO:tensorflow:loss = 0.016405549, step = 2151 (0.036 sec)
INFO:tensorflow:global_step/sec: 884.695
INFO:tensorflow:loss = 0.11575867, step = 2176 (0.028 sec)
INFO:tensorflow:global_step/sec: 880.587
INFO:tensorflow:loss = 0.8378404, step = 2201 (0.028 sec)
INFO:tensorflow:global_step/sec: 954.648
INFO:tensorflow:loss = 0.7383179, step = 2226 (0.026 sec)
INFO:tensorflow:global_step/sec: 948.354
INFO:tensorflow:loss = 0.026929017, step = 2251 (0.026 sec)
INFO:tensorflow:global_step/sec: 944.518
INFO:tensorflow:loss = 3.4697587, step = 2276 (0.026 sec)


INFO:tensorflow:global_step/sec: 696.784
INFO:tensorflow:loss = 7.307632, step = 4076 (0.035 sec)
INFO:tensorflow:global_step/sec: 893.988
INFO:tensorflow:loss = 0.25729635, step = 4101 (0.028 sec)
INFO:tensorflow:global_step/sec: 791.169
INFO:tensorflow:loss = 0.008135661, step = 4126 (0.032 sec)
INFO:tensorflow:global_step/sec: 839.297
INFO:tensorflow:loss = 0.41335812, step = 4151 (0.031 sec)
INFO:tensorflow:global_step/sec: 829.511
INFO:tensorflow:loss = 4.2915435e-06, step = 4176 (0.030 sec)
INFO:tensorflow:global_step/sec: 826.112
INFO:tensorflow:loss = 0.011010121, step = 4201 (0.030 sec)
INFO:tensorflow:global_step/sec: 790.746
INFO:tensorflow:loss = 0.0010929118, step = 4226 (0.032 sec)
INFO:tensorflow:global_step/sec: 828.135
INFO:tensorflow:loss = 0.0004363416, step = 4251 (0.029 sec)
INFO:tensorflow:global_step/sec: 777.143
INFO:tensorflow:loss = 0.01991529, step = 4276 (0.033 sec)
INFO:tensorflow:global_step/sec: 680.58
INFO:tensorflow:loss = 0.5841587, step = 4301 (0.039 